In [1]:

from langchain_core.documents import Document
document=[

Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [6]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq


In [5]:
embedding=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

In [7]:
model=ChatGroq(
     model_name="llama3-8b-8192" 
)

In [8]:
croma_db=Chroma.from_documents(documents=document,embedding=embedding)

In [9]:
croma_db.similarity_search("cat")

[Document(id='38a19414-594d-48da-95f3-c82b3bbe2636', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='d8c4e3e4-d1cc-4b00-a708-a2c61c6ccff3', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='6896db9d-904b-44b3-b6fd-8682b3f93233', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(id='59f4d22a-2624-453d-aa8f-ff7426a2d1cc', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [11]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever=RunnableLambda(croma_db.similarity_search).bind(k=1)

In [12]:
retriever.batch(["cat","dog"])

[[Document(id='38a19414-594d-48da-95f3-c82b3bbe2636', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='d8c4e3e4-d1cc-4b00-a708-a2c61c6ccff3', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
message="""
Answer this question using the provided context only
{question}
Context:
{context}"""

prompt=ChatPromptTemplate.from_messages([("human",message)])

In [17]:
rag_chain={"context":retriever,"question":RunnablePassthrough()}|prompt|model

In [22]:
response=rag_chain.invoke("tell me about cats?")
print(response.content)

According to the provided context, cats are independent pets that often enjoy their own space.
